In [1]:
#from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from nltk import FreqDist
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer


In [2]:
import csv
import nltk
import re
from nltk.tokenize import StanfordTokenizer
from nltk.tag import StanfordNERTagger
from nltk.tag import StanfordPOSTagger

In [3]:
fname = r"reviews_ChinesePN_5000.csv"
f = open(fname, 'r', encoding = 'utf8')
reader = csv.reader(f)

In [4]:
def separateTextTarget(reader):
    docs = []
    target = []    
    for row in reader:
        break
    for row in reader:
        docs.append(row[0])
        target.append(row[1])
    return (docs, target)

In [5]:
docs, target = separateTextTarget(reader)
f.close()

In [6]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer = StanfordTokenizer('../stanford-postagger/stanford-postagger.jar')
st = StanfordNERTagger("../stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz",
               "../stanford-ner/stanford-ner.jar")
sp = StanfordPOSTagger('../stanford-postagger/models/english-bidirectional-distsim.tagger',
					   '../stanford-postagger/stanford-postagger.jar',
					   encoding='utf-8')


In [14]:
train_data = docs[:4000]
test_data = docs[4000:]

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(min_df=3, stop_words ='english')

# Bags of words (BOWs): X_train_counts, X_test_counts
X_train_counts = count_vect.fit_transform(train_data) #**SLIGHLTLY DIFFERENT DIM (2989, 3966)
X_test_counts = count_vect.transform(test_data)

#Term Frequency times Inverse Document Frequency (tf-idf): X_train_tfidf, X_test_tfidf
# Compute tfidf feature values and store in 'X_train_tfidf' and 'X_test_tfidf'
tfidf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
#fit/compute Tfidf weights using X_train_counts
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#apply fitted weights to X_test_counts
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [37]:
text = " ".join(test_data)

In [45]:
a = "X_train_counts = count_vect.fit_transform(train_data)"
b = tokenizer.tokenize(a)

In [46]:
text2 = tokenizer.tokenize(text)

In [54]:
text3 = nltk.pos_tag(text2)

In [56]:
count_vect = CountVectorizer(min_df=3, stop_words ='english')
X_train_counts = count_vect.fit(text2)

In [61]:
from nltk.sentiment import SentimentAnalyzer


In [62]:
reviews = test_data

In [63]:
sta = SentimentAnalyzer()

In [74]:
import pandas as pd
train = pd.read_csv(r"reviews_ChinesePN_5000.csv", header = 0)

In [81]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [82]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["text"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( train["text"][i] ) )

In [83]:
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [85]:
train_data_features.shape #5000 rows, 5000 features

(5000, 5000)

In [92]:
vocab = vectorizer.get_feature_names()
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set


[ 97  24 174   4   9]
